<a href="https://colab.research.google.com/github/quotation3/TIL/blob/master/quotation_kim/NBC_%EC%8A%A4%ED%8C%B8%EB%A9%94%EC%9D%BC%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import pandas as pd
import numpy as np

In [71]:
text = [['me free lottery'],['free get free you'],['you free scholarship'],['free to contact me'],['you won award'],['you ticket lottery']]

In [72]:
text_split = []

for i in range(len(text)):
    text_split.append(' '.join(text[i]).split(' '))
mail_type = ['spam','spam','normal','normal','normal','spam']
data = np.array([text_split,mail_type])
df = pd.DataFrame(data.T,columns=['정제된단어들','분류'])
df


#     tokens.extend(text_split[i].split(' '))
# # tokens

,정제된단어들,분류
0,"[me, free, lottery]",spam
1,"[free, get, free, you]",spam
2,"[you, free, scholarship]",normal
3,"[free, to, contact, me]",normal
4,"[you, won, award]",normal
5,"[you, ticket, lottery]",spam


In [73]:
tokens = []
for i in range(len(text_split)):
    tokens.extend(text_split[i])
tokens = list(set(tokens))
tokens

['to',
 'award',
 'contact',
 'you',
 'free',
 'lottery',
 'me',
 'scholarship',
 'ticket',
 'won',
 'get']

In [74]:
spam_cnt_list = []
normal_cnt_list = []

for i in range(len(tokens)):
    spam_cnt = 0
    normal_cnt = 0
    for j in range(len(text_split)):    #tokens의 단어가 text_split[j]에 있으면 그 빈도 만큼 카운트 추가
        if tokens[i] in text_split[j]:
            if mail_type[j] == 'spam':
                spam_cnt+=text_split[j].count(tokens[i])
            elif mail_type[j] == 'normal':
                normal_cnt+=text_split[j].count(tokens[i])
            else:
                pass
    spam_cnt_list.append(spam_cnt)
    normal_cnt_list.append(normal_cnt)


In [94]:
p_spam = []
p_normal = []
for i in range(len(tokens)):
    prob_spam = (0.5+float(str(spam_cnt_list[i])))/(2*0.5+sum(spam_cnt_list))*100
    prob_normal = (0.5+float(str(normal_cnt_list[i])))/(2*0.5+sum(normal_cnt_list))*100
    p_spam.append("%0.2f" %prob_spam)
    p_normal.append("%0.2f" %prob_normal)

In [95]:
table = np.array([tokens,spam_cnt_list,normal_cnt_list,p_spam,p_normal])
pd.DataFrame(table.T, columns=['단어','spam','normal','P(w|spam)','P(w|normal)'])

,단어,spam,normal,P(w|spam),P(w|normal)
0,to,0,1,4.55,13.64
1,award,0,1,4.55,13.64
2,contact,0,1,4.55,13.64
3,you,2,2,22.73,22.73
4,free,3,2,31.82,22.73
5,lottery,2,0,22.73,4.55
6,me,1,1,13.64,13.64
7,scholarship,0,1,4.55,13.64
8,ticket,1,0,13.64,4.55
9,won,0,1,4.55,13.64


In [110]:
log_p_spam = []
log_p_normal = []
for i in range(len(tokens)):
     log_prob_spam = np.log((0.5+float(str(spam_cnt_list[i])))/(2*0.5+sum(spam_cnt_list)))
     log_prob_normal = np.log((0.5+float(str(normal_cnt_list[i])))/(2*0.5+sum(normal_cnt_list)))
     log_p_spam.append("%0.4f"%log_prob_spam)
     log_p_normal.append("%0.4f"%log_prob_normal)

log_table = np.array([tokens,log_p_spam,log_p_normal])
pd.DataFrame(log_table.T, columns=['단어','Log(P(w|spam))','Log(P(w|normal))'])

,단어,Log(P(w|spam)),Log(P(w|normal))
0,to,-3.0910,-1.9924
1,award,-3.0910,-1.9924
2,contact,-3.0910,-1.9924
3,you,-1.4816,-1.4816
4,free,-1.1451,-1.4816
5,lottery,-1.4816,-3.0910
6,me,-1.9924,-1.9924
7,scholarship,-3.0910,-1.9924
8,ticket,-1.9924,-3.0910
9,won,-3.0910,-1.9924


In [168]:
total_table = np.array([tokens,spam_cnt_list,normal_cnt_list,p_spam,p_normal,log_p_spam,log_p_normal])
pd.DataFrame(total_table.T, columns=['단어','spam','normal','P(w|spam)','P(w|normal)','Log(P(w|spam))','Log(P(w|normal))'])

,단어,spam,normal,P(w|spam),P(w|normal),Log(P(w|spam)),Log(P(w|normal))
0,to,0,1,4.55,13.64,-3.0910,-1.9924
1,award,0,1,4.55,13.64,-3.0910,-1.9924
2,contact,0,1,4.55,13.64,-3.0910,-1.9924
3,you,2,2,22.73,22.73,-1.4816,-1.4816
4,free,3,2,31.82,22.73,-1.1451,-1.4816
5,lottery,2,0,22.73,4.55,-1.4816,-3.0910
6,me,1,1,13.64,13.64,-1.9924,-1.9924
7,scholarship,0,1,4.55,13.64,-3.0910,-1.9924
8,ticket,1,0,13.64,4.55,-1.9924,-3.0910
9,won,0,1,4.55,13.64,-3.0910,-1.9924


In [175]:
#스팸일 확률과 정상일 확률을 구하는 함수 만들기
def spam_filtering (a,b):
    index_a = tokens.index(a)
    index_b = tokens.index(b)
    normal_prob = np.log(mail_type.count('normal')/len(mail_type))
    spam_prob = np.log(mail_type.count('spam')/len(mail_type))
    x1 = math.exp(float(log_p_normal[index_a])+(float(log_p_normal[index_b]))+(float("%0.4f"%normal_prob)))
    x2 = math.exp(float(log_p_spam[index_a])+(float(log_p_spam[index_b]))+(float("%0.4f"%spam_prob)))

    return ('{},{}가 포함된 메일이 스팸일 확률 : {}'.format(a,b,(x2/(x1+x2))*100), '{},{}가 포함된 메일이 정상일 확률 : {}'.format(a,b,(x1/(x1+x2))*100))

In [174]:
spam_filtering('free','lottery')

('free,lottery가 포함된 메일이 스팸일 확률 : 87.49988899428504',
 'free,lottery가 포함된 메일이 정상일 확률 : 12.500111005714952')